In [4]:
import string
import pandas as pd
from math import prod


with open('file.txt', 'r') as file:
    lines = file.readlines()

df = pd.read_csv('file.txt', sep=r'\s+', engine='python', 
                 header=None,  # No header row
                 on_bad_lines='warn')

print(df.head())

   0     1   2    3    4    5     6    7     8     9    ... 990  991  992 993  \
0  519   313  52  441   13   21    74  533  8519    94  ...  54   44    9   3   
1   23  3684  56  389  613   25   822  135   873  4851  ...  91   18  992  42   
2   15  1658  46  835  282  934  8292  761   416  5735  ...  38  514  345  63   
3   67  3259   4    8  926  925  3256  257    34  4557  ...  47  517  782  14   
4    *     +   +    +    +    *     +    *     +     +  ...   +    +    +   +   

   994 995 996  997 998 999  
0    5   2   8   89   8  54  
1   56   9   5   87   7  76  
2   82  15  21   28  86  43  
3  223  98  16  233  96  73  
4    *   *   *    *   *   +  

[5 rows x 1000 columns]


In [7]:
df_transposed = df.T  # Get fresh copy from original df

row_sums = []

for i, row in df_transposed.iterrows():
    operator = row[4]
    row_sum = 1 if operator == '*' else 0

    for j, col in enumerate(row):
        # Skip the operator column AND any existing row_sum column
        if j == 4 or (isinstance(col, str) and col == 'row_sum'):
            continue
            
        if str(col).isdigit() or (isinstance(col, (int, float)) and not pd.isna(col)):
            col_val = int(col) if str(col).isdigit() else int(col)
            
            if operator == '*':
                row_sum *= col_val
            else: 
                row_sum += col_val

    row_sums.append(row_sum)

# Add the row_sum column
df_transposed['row_sum'] = row_sums

total = df_transposed['row_sum'].sum()
print(f"\nGrand Total: {total}")


Grand Total: 3261038365331


In [15]:
# Check first few columns in detail
results = []
current_problem_numbers = []
current_operator = None

for col_idx, column in enumerate(zip(*lines)):
   
        
    # Check if all whitespace
    is_whitespace = all(c in string.whitespace for c in column)
    
    if is_whitespace:
        if current_operator and current_problem_numbers:
            if current_operator == '+':
                result = sum(current_problem_numbers)
            elif current_operator == '*':
                result = prod(current_problem_numbers)
            else:
                result = 0
            results.append(result)
        else:
            print(f'  -> Skipping (operator={current_operator}, numbers={current_problem_numbers})')
        
        # Reset
        current_problem_numbers.clear()
        current_operator = None
        continue
    
    # Extract content
    *number_chars, op_char = column
    
    # Check operator
    if op_char.strip() and op_char.strip() in "+*-/":
        current_operator = op_char.strip()
    
    # Extract number
    digit_chars = [c for c in number_chars if c in string.digits]
    if digit_chars:
        number = int(''.join(digit_chars))
        current_problem_numbers.append(number)

grand_total = sum(results)
print(f"\nGrand Total from detailed parsing: {grand_total}")



Grand Total from detailed parsing: 8342588838713
